# ACE Exploration

ACE (Advanced Composition Explorer) is equipped with nine scientific instruments to make comprehensive and coordinated in situ measurements. These instruments are categorized into two groups: High Resolution Spectrometers and Monitoring Instruments.

## High Resolution Spectrometers
- **CRIS** - Cosmic Ray Isotope Spectrometer
- **SIS** - Solar Isotope Spectrometer
- **ULEIS** - Ultra Low Energy Isotope Spectrometer
- **SEPICA** - Solar Energetic Particle Ionic Charge Analyzer
- **SWICS** - Solar Wind Ion Composition Spectrometer
- **SWIMS** - Solar Wind Ion Mass Spectrometer

## Monitoring Instruments
- **MAG** - Magnetic Field Monitor
- **SWEPAM** - Solar Wind Electron, Proton and Alpha Monitor
- **EPAM** - Electron, Proton and Alpha Monitor
- **SWICS** - Solar Wind Ion Composition Spectrometer

All open-source ACE data are formatted using hierarchical data format (HDF). The data are organized by instrument and by time-averaging periods. Each instrument's data are stored in separate HDF data files, and separate HDF files also contain the data from the different averaging periods. For most of the instruments, the data are averaged hourly, daily, and per 27 days (1 Bartels rotation).

## About Hierarchical Data Formats
Hierarchical Data Formats (HDF) are open source file formats that support large, complex, heterogeneous data. HDF files use a “file directory” like structure that allows you to organize data within the file in many different structured ways, as you might do with files on your computer. HDF files also allow for embedding of metadata making them self-describing.

---

## Analytical Questions
How can we apply novel dimension reduction methods, such as PCA, TSNE, etc., to obtain informative solar wind in-situ data representation in low-dimensional space? How can this low-dimensional representation provide better 2D/3D visualization support than traditional dimension reduction techniques?

## Libraries and global variables

In [56]:
# Standard library imports
import sys
import os

# Third-party imports
from contextlib import suppress
import warnings
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.decomposition import PCA, KernelPCA
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
# from keras.layers import Input, Dense
# from keras.models import Model

# Local application imports
sys.path.append("../src/scripts")
from utilities import (
    parse_hdf_data,
    merge_dataframes,
    missing_occurrences,
    sort_columns_except_key,
    visualize_flag,
    add_datetime_column,
)

# Set the warning filter to ignore all warnings
warnings.filterwarnings("ignore")

In [57]:
# global variables
MISSING_FLAG = -999.900
N_SPLITS = 4

## Data Import

In [58]:
# read data
data_dir = "../data/ace/raw"
swics_1hr_dir = f"{data_dir}/swics_1hr"
swics_2hr_dir = f"{data_dir}/swics_2hr"

mag_df = parse_hdf_data(f"{data_dir}/MAG_data_1hr.txt")
swepam_df = parse_hdf_data(f"{data_dir}/SWEPAM_data_1hr.txt")
epam_df = parse_hdf_data(f"{data_dir}/EPAM_data_1hr.txt")

swics_dfs = []
for dir in [swics_1hr_dir, swics_2hr_dir]:
    for file in os.listdir(dir):
        swics_dfs.append(parse_hdf_data(f"{dir}/{file}"))
swics_df = pd.concat(swics_dfs)

In [59]:
ACE_DATASETS = [mag_df, swepam_df, epam_df, swics_df]
ACE_DATASETS_NAMES = ["MAG", "SWEPAM", "EPAM", "SWICS"]

In [60]:
# dtype conversion
for df in ACE_DATASETS:
    df[["year", "day", "hr", "min", "sec"]] = df[
        ["year", "day", "hr", "min", "sec"]
    ].astype(int)

    with suppress(KeyError):
        df['Quality'] = df['Quality'].astype(str)

In [61]:
# datetime conversion and drop redundant features
for df in ACE_DATASETS:
    add_datetime_column(df).drop(
        columns=["year", "day", "hr", "min", "sec", "fp_year", "fp_doy"],
        inplace=True,
        axis=1,
    )

# swics_df may contain duplicate records to nature of 1.0 and 2.0 data collection
swics_df.drop_duplicates(subset="datetime", inplace=True)

## Data Cleaning

### Descriptives

In [11]:
for df, df_name in zip(ACE_DATASETS, ACE_DATASETS_NAMES):
    print(f"Dataframe: {df_name}")
    display(df.info())
    display(df.describe())
    print("\n" + ("-" * 20))

Dataframe: MAG
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231336 entries, 0 to 231335
Data columns (total 26 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   datetime       231336 non-null  datetime64[ns]
 1   ACEepoch       231336 non-null  float64       
 2   SCclock        231336 non-null  float64       
 3   Br             231336 non-null  float64       
 4   Bt             231336 non-null  float64       
 5   Bn             231336 non-null  float64       
 6   Bmag           231336 non-null  float64       
 7   Delta          231336 non-null  float64       
 8   Lambda         231336 non-null  float64       
 9   Bgse_x         231336 non-null  float64       
 10  Bgse_y         231336 non-null  float64       
 11  Bgse_z         231336 non-null  float64       
 12  Bgsm_x         231336 non-null  float64       
 13  Bgsm_y         231336 non-null  float64       
 14  Bgsm_z         231336 non-null  float

None

,ACEepoch,SCclock,Br,Bt,Bn,Bmag,Delta,Lambda,Bgse_x,Bgse_y,...,dBrms,sigma_B,fraction_good,N_vectors,pos_gse_x,pos_gse_y,pos_gse_z,pos_gsm_x,pos_gsm_y,pos_gsm_z
count,2.313360e+05,2.313360e+05,231336.000000,231336.000000,231336.000000,231336.000000,231336.000000,231336.000000,231336.000000,231336.000000,...,231336.000000,231336.000000,231336.000000,231336.000000,2.313360e+05,231336.000000,231336.000000,2.313360e+05,231336.000000,231336.000000
mean,4.674654e+08,4.205955e+08,-3.871007,-3.808264,-3.784931,1.979197,-3.869750,196.062699,-3.718902,-3.781131,...,-1.521951,-3.398425,0.995905,224.077368,1.482788e+06,-2437.617370,1683.444487,1.482804e+06,-1896.171801,4651.744091
std,2.404119e+08,2.752799e+08,61.575025,61.605103,61.550178,61.915690,67.842521,124.230072,61.584400,61.606392,...,61.575131,61.439070,0.062182,13.990851,6.929838e+04,182760.507192,106453.364131,6.932010e+04,176318.105686,116696.835180
min,5.106240e+07,6.970000e+02,-999.900000,-999.900000,-999.900000,-999.900000,-999.900000,-999.900000,-999.900000,-999.900000,...,-999.900000,-999.900000,0.000000,0.000000,3.442000e+04,-475680.000000,-165980.000000,3.442000e+04,-463990.000000,-265270.000000
25%,2.592639e+08,2.082025e+08,-2.659000,-2.532250,-1.317000,3.877750,-18.268250,122.090750,-2.522000,-2.513000,...,1.165000,0.156000,1.000000,225.000000,1.432400e+06,-182880.000000,-104232.500000,1.432400e+06,-169330.000000,-88805.000000
50%,4.674654e+08,4.164044e+08,-0.141500,0.010000,0.005000,5.077000,0.082000,178.055500,0.099000,-0.052000,...,1.895000,0.261000,1.000000,225.000000,1.480800e+06,-2113.150000,3099.200000,1.480800e+06,-2782.250000,10900.000000
75%,6.756669e+08,6.246062e+08,2.484000,2.481000,1.306000,6.851000,17.900000,300.733500,2.624000,2.484000,...,2.916000,0.460000,1.000000,225.000000,1.538900e+06,179700.000000,107750.000000,1.538900e+06,166180.000000,107210.000000
max,8.838684e+08,4.294967e+09,41.120000,35.083000,36.638000,71.993000,89.107000,359.998000,33.287000,45.473000,...,40.161000,28.231000,1.000000,226.000000,1.594800e+06,268000.000000,164090.000000,1.594800e+06,298140.000000,308840.000000



--------------------
Dataframe: SWEPAM
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204768 entries, 0 to 204767
Data columns (total 30 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   datetime                      204768 non-null  datetime64[ns]
 1   ACEepoch                      204768 non-null  float64       
 2   proton_density                204768 non-null  float64       
 3   proton_temp                   204768 non-null  float64       
 4   He4toprotons                  204768 non-null  float64       
 5   proton_speed                  204768 non-null  float64       
 6   x_dot_GSE                     204768 non-null  float64       
 7   y_dot_GSE                     204768 non-null  float64       
 8   z_dot_GSE                     204768 non-null  float64       
 9   x_dot_RTN                     204768 non-null  float64       
 10  y_dot_RTN                     204768 non

None

...

,ACEepoch,P1,P2,P3,P4,P5,P6,P7,P8,unc_P1,...,FP6,FP7,unc_E1,unc_E2,unc_E3,unc_E4,unc_FP5,unc_FP6,unc_FP7,livetime
count,2.323510e+05,2.323510e+05,2.323510e+05,2.323510e+05,232351.000000,232351.000000,232351.000000,232351.000000,232351.000000,232351.000000,...,2.323510e+05,232351.000000,232351.000000,232351.000000,232351.000000,232351.000000,232351.000000,232351.000000,232351.000000,232351.000000
mean,4.707360e+08,1.039626e+03,8.344632e+02,-1.399470e+01,-491.177701,-639.511981,-714.147444,46.249265,9.983937,-841.980513,...,3.289076e+31,14.952518,-980.091116,-980.091059,-980.091038,-3.066417,-3.060326,-3.045570,-3.052071,3327.123985
std,2.414668e+08,2.967206e+04,3.329843e+04,1.205079e+04,4048.188829,2237.219437,1142.555217,631.786597,222.343196,364.647106,...,1.585427e+34,275.195568,139.337412,139.337814,139.337962,55.460276,55.460615,55.422628,55.499859,256.586417
min,5.250600e+07,-9.999000e+02,-9.999000e+02,-9.999000e+02,-999.900000,-999.900000,-999.900000,-999.900000,-999.900000,-999.900000,...,-9.999000e+02,-999.900000,-999.900000,-999.900000,-999.900000,-999.900000,-999.900000,-999.900000,-999.900000,-999.900000
25%,2.616210e+08,-9.999000e+02,-9.999000e+02,-9.999000e+02,-999.900000,-999.900000,-999.900000,0.158420,0.031849,-999.900000,...,1.081200e+00,0.078608,-999.900000,-999.900000,-999.900000,0.014030,0.018090,0.022730,0.022490,3324.000000
50%,4.707360e+08,-9.999000e+02,-9.999000e+02,-9.999000e+02,-999.900000,-999.900000,-999.900000,0.222130,0.042992,-999.900000,...,1.396500e+00,0.102080,-999.900000,-999.900000,-999.900000,0.021640,0.028310,0.041120,0.043930,3351.000000
75%,6.798510e+08,-9.999000e+02,-9.999000e+02,-9.999000e+02,-999.900000,-999.900000,-999.900000,1.352600,0.208750,-999.900000,...,4.447700e+00,0.378690,-999.900000,-999.900000,-999.900000,0.024300,0.032250,0.046860,0.050210,3380.000000
max,8.889660e+08,2.781500e+06,3.976300e+06,1.072400e+06,289950.000000,214950.000000,119990.000000,67757.000000,28195.000000,0.189000,...,7.642200e+36,32541.000000,0.093250,0.102100,0.142900,0.447200,0.378000,1.000000,0.426400,3462.000000



--------------------
Dataframe: SWICS
<class 'pandas.core.frame.DataFrame'>
Int64Index: 158627 entries, 0 to 2563
Data columns (total 70 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   datetime     158627 non-null  datetime64[ns]
 1   ACEepoch     158627 non-null  float64       
 2   nHe2         109672 non-null  float64       
 3   nHe2_err     109672 non-null  float64       
 4   vHe2         158627 non-null  float64       
 5   vthHe2       158627 non-null  float64       
 6   vHe2_err     158627 non-null  float64       
 7   vthHe2_err   158627 non-null  float64       
 8   qf_He        158627 non-null  float64       
 9   vC5          109672 non-null  float64       
 10  vthC5        109672 non-null  float64       
 11  vC5_err      109672 non-null  float64       
 12  vthC5_err    109672 non-null  float64       
 13  qf_C5        109672 non-null  float64       
 14  vO6          109672 non-null  float64       
 1

None

,ACEepoch,nHe2,nHe2_err,vHe2,vthHe2,vHe2_err,vthHe2_err,qf_He,vC5,vthC5,...,qf_NetoO,MgtoO,MgtoO_err,qf_MgtoO,SitoO,SitoO_err,qf_SitoO,O8to6,O8to6_err,qf_O8to6
count,1.586270e+05,109672.000000,109672.000000,158627.000000,158627.000000,1.586270e+05,1.586270e+05,158627.000000,109672.000000,109672.000000,...,8695.000000,8695.000000,8695.000000,8695.000000,8695.000000,8695.000000,8695.000000,48955.000000,48955.000000,48955.000000
mean,4.112591e+08,-301.459735,-301.623230,87.638222,-305.918902,-9.999900e+03,-9.999900e+03,-0.031186,52.110503,-349.563415,...,0.005750,-48.142825,-48.298045,0.007476,-52.722181,-52.898641,0.011041,-442.001870,-438.643672,-0.040466
std,2.241762e+08,1710.366953,1710.338113,1894.146280,1803.955726,3.637990e-12,3.637990e-12,0.251466,2001.469757,1918.414396,...,0.262202,693.371588,693.360769,0.273539,725.472285,725.459412,0.295457,2055.484937,2054.715604,0.197051
min,6.609656e+07,-9999.900000,-9999.900000,-9999.900000,-9999.900000,-9.999900e+03,-9.999900e+03,-1.000000,-9999.900000,-9999.900000,...,-1.000000,-9999.900000,-9999.900000,-1.000000,-9999.900000,-9999.900000,-1.000000,-9999.900000,-9999.900000,-1.000000
25%,2.293928e+08,0.060517,0.000266,351.100000,18.145000,-9.999900e+03,-9.999900e+03,0.000000,351.537500,17.267750,...,0.000000,0.115040,0.002677,0.000000,0.137785,0.002768,0.000000,0.002583,2.203600,0.000000
50%,3.841231e+08,0.124300,0.000386,411.940000,27.646000,-9.999900e+03,-9.999900e+03,0.000000,417.625000,30.825000,...,0.000000,0.144400,0.003699,0.000000,0.169760,0.003776,0.000000,0.008336,2.664400,0.000000
75%,5.852564e+08,0.219200,0.000540,502.785000,38.149000,-9.999900e+03,-9.999900e+03,0.000000,514.612500,42.328250,...,0.000000,0.186580,0.005508,0.000000,0.211245,0.005368,0.000000,0.021736,3.418100,0.000000
max,8.722127e+08,2.319100,0.002424,1844.900000,201.000000,-9.999900e+03,-9.999900e+03,16.000000,1877.300000,222.290000,...,8.000000,1.879000,0.163780,8.000000,1.442300,0.161400,8.000000,1.598100,4.166100,0.000000



--------------------


### Retain *Good* Quality data

Good data is flagged by the researchers with a value of 0. 

In [62]:
for c, (df, df_name) in enumerate(zip(ACE_DATASETS, ACE_DATASETS_NAMES)):

    with suppress(KeyError):  # not all datasets have the quality flag
        if df_name != "SWICS":
            df = df[df["Quality"] == "0.0"]
            df.drop(columns=["Quality"], inplace=True, axis=1)
        else:
            qf_cols = swics_df.filter(regex="^qf_").columns
            df = swics_df[
                (swics_df[qf_cols].isna() | swics_df[qf_cols].eq(0)).any(axis=1)
            ]

    ACE_DATASETS[c] = df

mag_df, swepam_df, epam_df, swics_df = ACE_DATASETS

### Join data

In [40]:
# find unique timestamps
mag_dates, swepam_dates, epam_dates, swics_dates = [
    df.datetime.unique() for df in ACE_DATASETS
]

# find the common dates for 1hr interval data
common_dates_1hr = reduce(
    np.intersect1d, (mag_dates, swepam_dates, epam_dates)
)

# find the common dates for 2hr interval data
common_date_2hr = reduce(
    np.intersect1d, (mag_dates, swepam_dates, epam_dates, swics_dates)
    )

print(len(common_dates_1hr))
print(len(common_date_2hr))

187272
139207


In [63]:
# join the 1hr to 2 hr interval datasets
insitu_df = merge_dataframes(ACE_DATASETS, "datetime")
df = sort_columns_except_key(insitu_df, "datetime")


### Handling Missing Values

Missing data has the value of -999.900. Assert that there are no longer missing values due to dropping data labeled as not of good quality.

In [ ]:
df_missing = missing_occurrences(df, MISSING_FLAG).sort_values(
    ascending=False, by="Missing_Count"
)
visualize_flag(df_missing)
df.shape

#### Imputation Methods 

Univariate time series imputation methods: 
- Mean (median)
- Last Observed Carried Forward
- Linear Interpolation
- Polynomial Interpolation
- Kalman Filter
- Moving Averge
- Random

Multivariate Time Series Imputation methods. 
- K-Nearest Neighbords
- Random Forest
- Multiple Singular Spectral Analysis
- Expectation-Maximization
- Multiple Imputation with Chained Equations

In [ ]:
# Prepare for imputation
numeric_df = df.select_dtypes(include=[np.number])
df = df.replace(-9999.9, np.nan)
X_incomplete = df.select_dtypes(include=[np.number])

# KNN Imputer with 3 neighbors
knn_imputer = KNNImputer(n_neighbors=3, add_indicator=True)
knn_imputed_data = knn_imputer.fit_transform(X_incomplete)
df_knn_imputed = pd.DataFrame(knn_imputed_data, columns=numeric_df.columns)
df_knn_imputed.insert(0, "datetime", df["datetime"])

# Create the Random Forest imputer
rf_imputer = IterativeImputer(
    estimator=RandomForestRegressor(), random_state=0, max_iter=10
)
rf_imputed_data = rf_imputer.fit_transform(X_incomplete)
df_knn_imputed = pd.DataFrame(rf_imputed_data, columns=numeric_df.columns)
df_knn_imputed.insert(0, "datetime", df["datetime"])

In [ ]:
# Create the Random Forest imputer
rf_imputer = IterativeImputer(
    estimator=RandomForestRegressor(), random_state=0, max_iter=10
)

# Perform the imputation
imputed_data = rf_imputer.fit_transform(X_incomplete)

## Data Transformation

### Log transformation of all quantities

In [ ]:
# TODO: select the subset that needs log transformation
df.apply(np.log10)

## Dimensionality Reduction

### Dimensionality Reduction Using PCA

In [ ]:
# Fit PCA
pca = PCA().fit(X)

# Plot the explained variances
features = range(pca.n_components_)
plt.bar(features, pca.explained_variance_ratio_, color="black")
plt.xlabel("PCA features")
plt.ylabel("variance %")
plt.xticks(features)

# Save components to a DataFrame
PCA_components = pd.DataFrame(pca.transform(X))

plt.show()

In [ ]:
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
num_components = np.where(cumulative_variance > 0.95)[0][0] + 1
print("Number of components to explain 95% Variance: ", num_components)

In [ ]:
# Create a PCA that will retain 2 components
pca = PCA(n_components=num_components, whiten=True)

# Conduct PCA
X_pca = pca.fit_transform(X)

# Show the new data
print("original shape:   ", X.shape)
print("transformed shape:", X_pca.shape)

# The transformed data has been reduced to two dimensions
df = pd.DataFrame(
    data=X_pca,
)
print(df.head())

### Dimensionality Reduction Using Kernel PCA

In [ ]:
# Fit Kernel PCA with n_components=None to compute all components
kpca = KernelPCA(n_components=None, kernel="rbf")
kpca.fit(X)

# Get eigenvalues
eigenvalues = kpca.lambdas_

# Plot eigenvalues
plt.plot(eigenvalues, "bo-")
plt.xlabel("Index")
plt.ylabel("Eigenvalue")
plt.show()

In this plot, the x-axis represents the index of each component (in descending order of eigenvalue), and the y-axis represents the corresponding eigenvalue. You typically choose the number of components at the point where adding another component doesn't significantly increase the eigenvalue (the "elbow" of the plot).

### Dimensionality Reduction Using Autoencoders

In [ ]:
# Define the size of the encoded representation
encoding_dim = 2  # 2-dimensional encoded representation

# Define the input layer
input_img = Input(shape=(X.shape[1],))

# Define the encoded layer
encoded = Dense(encoding_dim, activation="relu")(input_img)

# Define the decoded layer
decoded = Dense(X.shape[1], activation="sigmoid")(encoded)

# Define the autoencoder model
autoencoder = Model(input_img, decoded)

# Define the encoder model
encoder = Model(input_img, encoded)

# Define the decoder model
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

# Compile the autoencoder
autoencoder.compile(optimizer="adadelta", loss="binary_crossentropy")

# Train the autoencoder
autoencoder.fit(X, X, epochs=50, batch_size=256, shuffle=True)

# Use the encoder to reduce the dimensionality of the data
X_encoded = encoder.predict(X)

print("original shape:   ", X.shape)
print("transformed shape:", X_encoded.shape)

## Self-Organizing Maps